# Segmenting and Clustering Neighborhoods in Toronto

This notebook aims to cluster the neighborhoods around Toronto using k-means clustering method. 

The postal code data was scraped online from the Wikipedia page of Canada, and the geolocation data was retrieved using Foursquare API. The data was preprocessed/ wrangled for analysis. The data was then clustered using k-means clustering. 

## Part 1. Retreiving the Data Online

In [1]:
# !pip install beautifulsoup4
# !pip install requests

In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
from pandas import DataFrame

### 1. Get data from Wikipedia and convert to bs4 object

In [3]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [4]:
soup = BeautifulSoup(url, 'html.parser')

### 2. Scrape data from table to lists

In [5]:
# store the postal code, borough and neighborhood in lists

postalcode = []
borough = []
neighborhood = []

for row in soup.find('table').find('tbody').find_all('tr'):
    for col in row.find_all('td'):
        if col.span.text == 'Not assigned': # ignore rows with 'not assigned'
            pass
        else:
            postalcode.append(col.find('b').text) 
            cell = col.span.text.split('(')
            borough.append(cell[0])
            neighborhood.append(cell[1].replace(")","").strip(" ").replace('/',','))
        
# checking if all lists have the same number of entries
print(len(postalcode),len(borough),len(neighborhood))


103 103 103


### 3. Convert into dataframe

In [6]:
# converting data into dataframe

lst = [postalcode, borough, neighborhood]

df = DataFrame(lst).transpose()
df.columns = ['Postal Code', 'Borough', 'Neighborhood']
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


Checking if there are any 'Not assigned' under the 'Neighborhood' column

In [7]:
df.loc[df['Neighborhood'] == "Not assigned"]

,Postal Code,Borough,Neighborhood


In [8]:
# shape of dataframe
df.shape

(103, 3)

## Part 2. Getting Geolocation Data

In [9]:
#!pip install geocoder

Attempt to use geocoder API:

In [10]:
import geocoder

In [11]:
# # initialize your variable to None
# lat_lng_coords = None

# # test out one postalcode
# postal_code = 'M7A'

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#     g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#     print(g.latlng)
#     lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

Realized that geocoder isn't working; import csv file instead

In [12]:
geodata = pd.read_csv('Geospatial_Coordinates.csv')
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the postal code and geodata datasets

In [13]:
toronto_data = pd.merge(df, geodata, how='inner', on='Postal Code')
toronto_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509


## Part 3. Data Exploration and Analysis

In [14]:
# !conda install -c conda-forge geopy --yes
# !pip install -U scikit-learn
# !pip install folium==0.5.0

In [15]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage

from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 1. Plot out neighborhoods in Toronto

In [16]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [18]:
# create map of Toronto using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color= 'blue',
        fill=True,
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### 2. Consider only one borough

I will choose all boroughs containing the **Toronto** to be more focused on the neighbors I am interested in analysing.

In [19]:
all_toronto_data = toronto_data.loc[toronto_data['Borough'].str.contains("Toronto", case=False)]
all_toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


#### Foursquare credentials

In [20]:
import config as cfg

In [21]:
CLIENT_ID = cfg.foursquarekey['client_id'] # your Foursquare ID
CLIENT_SECRET = cfg.foursquarekey['client_secret'] # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails are in!')

Your credentails are in!


### 3. Explore neighborhoods in Toronto

Let us find out the number and types of venues around Toronto.

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
all_toronto_venues = getNearbyVenues(names = all_toronto_data['Neighborhood'], latitudes= all_toronto_data['Latitude'],longitudes= all_toronto_data['Longitude'], radius=1000)

In [24]:
print(all_toronto_venues.shape)
all_toronto_venues.head()

(3173, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park , Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park , Harbourfront",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


In [25]:
all_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"Brockton , Parkdale Village , Exhibition Place",100,100,100,100,100,100
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",15,15,15,15,15,15
Central Bay Street,100,100,100,100,100,100
Christie,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100
"Commerce Court , Victoria Hotel",100,100,100,100,100,100
Davisville,100,100,100,100,100,100
Davisville North,100,100,100,100,100,100


In [26]:
print('There are {} uniques categories.'.format(len(all_toronto_venues['Venue Category'].unique())))

There are 274 uniques categories.


### 4. Analyse each neighborhood

Determine what are the common venues in each neighborhood.

In [27]:
# one hot encoding
all_toronto_onehot = pd.get_dummies(all_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
all_toronto_onehot['Neighborhood'] = all_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [all_toronto_onehot.columns[-1]] + list(all_toronto_onehot.columns[:-1])
all_toronto_onehot = all_toronto_onehot[fixed_columns]

all_toronto_onehot.head()

,Zoo,Accessories Store,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0

In [28]:
all_toronto_onehot.shape

(3173, 274)

In [29]:
all_toronto_grouped = all_toronto_onehot.groupby('Neighborhood').mean().reset_index()
all_toronto_grouped.head()

,Neighborhood,Zoo,Accessories Store,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.0,0.00,0.000000,0.000000,0.01,0.0,0.0,0.0,0.0,0.0,0.02,0.00,0.00,0.0,0.00,0.0,0.01,0.0,0.02,0.00,0.00,0.0,0.0

In [30]:
all_toronto_grouped.shape

(39, 274)

The following shows the top venues for each neighborhood in Toronto. 

In [31]:
num_top_venues = 6

for hood in all_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = all_toronto_grouped[all_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0          Coffee Shop  0.11
1                 Café  0.07
2                 Park  0.05
3                Hotel  0.05
4  Japanese Restaurant  0.04
5             Beer Bar  0.03


----Brockton , Parkdale Village , Exhibition Place----
                    venue  freq
0                    Café  0.07
1             Coffee Shop  0.06
2                     Bar  0.06
3              Restaurant  0.05
4  Furniture / Home Store  0.04
5                  Bakery  0.04


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
              venue  freq
0   Harbor / Marina  0.13
1              Café  0.13
2       Coffee Shop  0.13
3  Sculpture Garden  0.07
4            Garden  0.07
5    Scenic Lookout  0.07


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.04
2  Japanese Restaurant  0.03
3      Bubble Tea Shop  0.03
4                

Now, to tabulate the most common venues into categories:

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = all_toronto_grouped['Neighborhood']

for ind in np.arange(all_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(all_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Park,Hotel,Japanese Restaurant,Beer Bar,Restaurant,Pub,Creperie,Art Gallery
1,"Brockton , Parkdale Village , Exhibition Place",Café,Coffee Shop,Bar,Restaurant,Furniture / Home Store,Bakery,Gift Shop,Tibetan Restaurant,Performing Arts Venue,Soccer Stadium
2,"CN Tower , King and Spadina , Railway Lands , ...",Harbor / Marina,Café,Coffee Shop,Sculpture Garden,Garden,Scenic Lookout,Track,Dog Run,Park,Dance Studio
3,Central Bay Street,Coffee Shop,Café,Japanese Restaurant,Bubble Tea Shop,Park,Ramen Restaurant,Yoga Studio,Cosmetics Shop,Sushi Restaurant,Burrito Place
4,Christie,Korean Restaurant,Coffee Shop,Café,Grocery Store,Pizza Place,Mexican Restaurant,Cocktail Bar,Indian Restaurant,Sandwich Place,Bar


### 5. Clustering neighborhoods

Number of clusters has been set based on the best way to segregate the data (ie. no one cluster dominates). 

In [34]:
# set number of clusters
kclusters = 3

all_toronto_grouped_clustering = all_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(all_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 0, 0, 0])

In [35]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

all_toronto_merged = all_toronto_data

# merge all_toronto_merged with all_toronto_data to add latitude/longitude for each neighborhood
all_toronto_merged = all_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='right')

all_toronto_merged.head() 

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Café,Park,Hotel,Japanese Restaurant,Beer Bar,Restaurant,Pub,Creperie,Art Gallery
43,M6K,West Toronto,"Brockton , Parkdale Village , Exhibition Place",43.636847,-79.428191,1,Café,Coffee Shop,Bar,Restaurant,Furniture / Home Store,Bakery,Gift Shop,Tibetan Restaurant,Performing Arts Venue,Soccer Stadium
87,M5V,Downtown Toronto,"CN Tower , King and Spadina , Railway Lands , ...",43.628947,-79.394420,1,Harbor / Marina,Café,Coffee Shop,Sculpture Garden,Garden,Scenic Lookout,Track,Dog Run,Park,Dance Studio
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Café,Japanese Restaurant,Bubble Tea Shop,Park,Ramen Restaurant,Yoga Studio,Cosmetics Shop,Sushi Restaurant,Burrito Place
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1,Korean Restaurant,Coffee Shop,Café,Grocery Store,Pizza Place,Mexican Restaurant,Cocktail Bar,Indian Restaurant,Sandwich Place,Bar


Plotting out the neighborhoods of Toronto, categorized into their clusters:

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(all_toronto_merged['Latitude'], all_toronto_merged['Longitude'], all_toronto_merged['Neighborhood'], all_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if cluster == np.nan: 
        continue
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 6. Determine the characteristic of each cluster

We will consider the top three most common venues in characterizing each cluster.

##### The first cluster (red) seems to have numerous coffee shops and cafes, but also has a wide variety of other types of venues.

In [37]:
cluster_1 = all_toronto_merged.loc[all_toronto_merged['Cluster Labels'] == 0, all_toronto_merged.columns[[1] + list(range(5, all_toronto_merged.shape[1]))]]
print(cluster_1.value_counts('1st Most Common Venue'), '\n')
print(cluster_1.value_counts('2nd Most Common Venue'), '\n')
print(cluster_1.value_counts('3rd Most Common Venue'), '\n')
cluster_1.head()

1st Most Common Venue
Coffee Shop           6
Café                  2
Italian Restaurant    2
Park                  2
Indian Restaurant     1
Pub                   1
Sushi Restaurant      1
dtype: int64 

2nd Most Common Venue
Coffee Shop           5
Café                  2
Italian Restaurant    2
Park                  2
Breakfast Spot        1
Grocery Store         1
Restaurant            1
Sushi Restaurant      1
dtype: int64 

3rd Most Common Venue
Coffee Shop           3
Bank                  2
Bakery                1
Brewery               1
Café                  1
Grocery Store         1
Italian Restaurant    1
Park                  1
Pizza Place           1
Skating Rink          1
Sushi Restaurant      1
Theater               1
dtype: int64 



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
79,Central Toronto,0,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Café,Restaurant,Indian Restaurant,Middle Eastern Restaurant,Dessert Shop,Gym
67,Central Toronto,0,Coffee Shop,Italian Restaurant,Café,Restaurant,Gym,Sushi Restaurant,Dessert Shop,Pizza Place,Pharmacy,Sandwich Place
31,West Toronto,0,Café,Coffee Shop,Park,Italian Restaurant,Sushi Restaurant,Bar,Grocery Store,Brewery,Bakery,Furniture / Home Store
100,East TorontoBusiness reply mail Processing Cen...,0,Park,Coffee Shop,Brewery,Pizza Place,Italian Restaurant,Sushi Restaurant,Fast Food Restaurant,Electronics Store,Burrito Place,Steakhouse
68,Central Toronto,0,Park,Café,Bank,Coffee Shop,Skating Rink,Italian Restaurant,Japanese Restaurant,Playground,Burger Joint,Pharmacy


##### The second cluster (purple) dominates in coffee shops and cafes, with much less variation in other types of venues.

In [38]:
cluster_2 = all_toronto_merged.loc[all_toronto_merged['Cluster Labels'] == 1, all_toronto_merged.columns[[1] + list(range(5, all_toronto_merged.shape[1]))]]
print(cluster_2.value_counts('1st Most Common Venue'), '\n')
print(cluster_2.value_counts('2nd Most Common Venue'), '\n')
print(cluster_2.value_counts('3rd Most Common Venue'), '\n')
cluster_2.head()

1st Most Common Venue
Coffee Shop          13
Café                  6
Gastropub             1
Greek Restaurant      1
Harbor / Marina       1
Korean Restaurant     1
dtype: int64 

2nd Most Common Venue
Café                   11
Coffee Shop             4
Bar                     2
Japanese Restaurant     2
Bakery                  1
Brewery                 1
Diner                   1
Greek Restaurant        1
dtype: int64 

3rd Most Common Venue
Hotel                  4
Bar                    3
Japanese Restaurant    3
Café                   2
Coffee Shop            2
Italian Restaurant     2
Park                   2
Restaurant             2
American Restaurant    1
Convenience Store      1
Gastropub              1
dtype: int64 



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Downtown Toronto,1,Coffee Shop,Café,Park,Hotel,Japanese Restaurant,Beer Bar,Restaurant,Pub,Creperie,Art Gallery
43,West Toronto,1,Café,Coffee Shop,Bar,Restaurant,Furniture / Home Store,Bakery,Gift Shop,Tibetan Restaurant,Performing Arts Venue,Soccer Stadium
87,Downtown Toronto,1,Harbor / Marina,Café,Coffee Shop,Sculpture Garden,Garden,Scenic Lookout,Track,Dog Run,Park,Dance Studio
24,Downtown Toronto,1,Coffee Shop,Café,Japanese Restaurant,Bubble Tea Shop,Park,Ramen Restaurant,Yoga Studio,Cosmetics Shop,Sushi Restaurant,Burrito Place
25,Downtown Toronto,1,Korean Restaurant,Coffee Shop,Café,Grocery Store,Pizza Place,Mexican Restaurant,Cocktail Bar,Indian Restaurant,Sandwich Place,Bar


##### The third cluster (teal) is the only cluster with parks as its most common venue. 

In [39]:
all_toronto_merged.loc[all_toronto_merged['Cluster Labels'] == 2, all_toronto_merged.columns[[1] + list(range(5, all_toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,2,Park,Trail,Bus Line,Bookstore,Café,College Quad,Coffee Shop,Gym / Fitness Center,College Gym,Pakistani Restaurant


# Summary

To summarize, the Toronto neighborhoods were clustered into three major clusters: coffee shop/ cafe heavy areas, coffee shop/cafe light areas, and parks. 

### --------- END OF REPORT ---------